# 1. Setup

In [1]:
# some imports
import sys
import os
import MCMC_Noddy as mcmc     
from glob import glob
import vedo as vtkP

# give correct permissions to the executable
folder = os.getcwd()
noddyEXE = folder+'/noddy_linux.exe'
strV = 'chmod 777 '+noddyEXE
os.system(strV)

0

# 2. Choose hyper parameters

In [ ]:
HyperParameters = {}

HyperParameters['ScenarioNum'] = 18
HyperParameters['SimulationShiftType'] = 'Median Datum Shift'
HyperParameters['DatNormMethod'] = 'MedianInitialRounds'
HyperParameters['ExplorationRate'] = 'LinearErrorBased'
HyperParameters['ErrorType'] = 'Global'
HyperParameters['ErrorNorm'] = 'L1'
HyperParameters['ExplorationStage'] = 'Explore'
HyperParameters['AcceptProbType'] = 'Track Acceptance'
HyperParameters['cubesize'] = 150
HyperParameters['AcceptanceGoal'] = 0.2
HyperParameters['ConstNormFactor'] = 0.01
HyperParameters['GlobalMoveEachDir'] = 500
HyperParameters['XYZ_Axes_StepStd'] = 100
HyperParameters['Dip_StepStd'] = 5
HyperParameters['Slip_StepStd'] = 50
HyperParameters['DipDirection_StepStd'] = 6
HyperParameters['SlipParam'] = 0.1
HyperParameters['SteppingSizeMult'] = 1/0.9
HyperParameters['MaxFaultMarkerError'] = 525
HyperParameters['AmplitudeRatioChange'] = 0.1
HyperParameters['AzimuthMoveEachDirection'] = 6
HyperParameters['AxisRatioChange'] = 0.1
HyperParameters['DipMoveEachDirection'] = 32
HyperParameters['nExploreRuns'] = 50
HyperParameters['MO_WeightingMethod'] = 'Proportions'
HyperParameters['MCMC_SwitchWeightFreq'] = 20
HyperParameters['LocalWeightsMode'] = 'na'

Thread_num = 0 
HyperParameters['thread_num'] = Thread_num        
HyperParameters['Toy']=False
HyperParameters['verbose']=True
HyperParameters['nruns']=200
HyperParameters['BaseFolder']='Combo_Scratch'
HyperParameters['GeneralPerturbStrategy']='OnlyGlobal'
HyperParameters['ControlPointMovementUnits'] = 'Absolute value'
HyperParameters['errCalcMethodFaultMarker']= 'Distance'
HyperParameters['DataTypes'] = ['Grav', 'Mag', 'GT', 'FaultMarkers']
HyperParameters['JoinType']='LINES'
HyperParameters['xy_origin']=[316448, 4379166, -2700]
HyperParameters['xy_extent'] = [8850, 9000,3900]
HyperParameters['DatNormCoef'] = {'Grav': 2.4, 'Tracer': 1.0, 
                        'FaultMarkers': 500, 'GT': 315, 'Mag':300}
HyperParameters['verbose']=True    
HyperParameters['graniteIdx'] = 4
HyperParameters['Windows'] = False
HyperParameters['jupyter'] = True

HyperParameters['OutputImageFreq'] = 15        
HyperParameters['OptimMethod']='MCMC'

# 3. Run and view results every x times (x=output image frequency)

In [ ]:
if(HyperParameters['OptimMethod']=='MCMC'):
    mcmc.MCMC_Noddy(HyperParameters)

# 4. Find the model with least error

In [ ]:
#Loop through all of the files and take the best and and another 5 random ones and copy them over
def GetIterationNumFromFile(file):
    result = re.search('G_(.*)_Err', file)
    return int(result.group(1))

def GetErrFromFile(file):
    result = re.search('Err_(.*).his', file)
    return float(result.group(1))

folder = os.getcwd()+'/Combo_Scratch/Thread'+str(Thread_num)+'/HistoryFileInspection/'
files = glob(folder+'*.his')
nFiles = len(files)

MasterFileList = []
ErrV=1000000.0
bestIter=0
for i in range(nFiles):
    file_i = files[i]
    print('reading: '+ file_i)
    Err=GetErrFromFile(file_i)
    Iter=GetIterationNumFromFile(file_i)
    if(Err<ErrV):
        ErrV = Err
        bestIter = Iter
        best_model_file = file_i

print('best file is: ' + best_model_file)

# 5. Plot in 3D

In [10]:
import plot3d_util as plt3d
import vedo as vtkP
import pandas as pd
from scipy.spatial import Delaunay
vtkP.settings.embedWindow('k3d') #you can also choose to change to itkwidgets, k3d

In [25]:
#Alter the mesh size if desiring to speed up the process. Recommended size is 100
output_name = 'noddy_out'
cubesize = 100
includeGravityCalc = 0
xy_origin=[316448, 4379166, -2700]

plot = vtkP.Plotter(axes=1, bg='white', interactive=1)
plt3d.plot_3d_model(best_model_file, cubesize, plot, xy_origin=[316448, 4379166, -2700])

# add topography
##################
# perform a 2D Delaunay triangulation to get the cells from the point cloud
landSurfacePD = pd.read_csv("Data/TopographyNew2.csv")
landSurfacePD = landSurfacePD[['X', 'Y', 'rvalue_1']].values
tri = Delaunay(landSurfacePD[:, 0:2])

# create a mesh object for the land surface
landSurface = vtkP.Mesh([landSurfacePD, tri.simplices])

# in order to color it by the elevation, we use the z values of the mesh
zvals = landSurface.points()[:, 2]
landSurface.pointColors(zvals, cmap="terrain", vmin=1000)
landSurface.name = "Land Surface" # give the object a name

plot+=landSurface

plot.show(viewup='z')

Calculation time took 5.0256569385528564 seconds
Parsing time took 6.906073331832886 seconds
The number of triangle elements (cells/faces) is: 200305
Convert 2 VTK time took 0.594963550567627 seconds


Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[327825.97439773346…